In [0]:
import urllib.request
urllib.request.urlretrieve("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz",
                           "/tmp/kddcup_data.gz")
dbutils.fs.mv("file:/tmp/kddcup_data.gz", "dbfs:/kdd/kddcup_data.gz")
display(dbutils.fs.ls("dbfs:/kdd"))

path,name,size
dbfs:/kdd/kddcup_data.gz,kddcup_data.gz,2144903


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.appName("Intrusion_detection").getOrCreate()

dataRDD = spark.sparkContext.textFile("dbfs:/kdd/kddcup_data.gz", 4)

dataRDD.take(10)

Out[113]: ['0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,9,9,1.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,19,19,1.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,235,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,29,29,1.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,219,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.00,0.00,0.00,0.00,1.00,0.00,0.00,39,39,1.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,217,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.00,0.00,0.00,0.00,1.00,0.00,0.00,49,49,1.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,217,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.00,0.00,0.00,0.00,1.00,0.00,0.00,59,59,1.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,212,1940,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,2,0.00,0.00,0.00,0.00,1.00,0.00,1.00,1,69,1.00,0.00,1.00,0.04,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,159,4087,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.00,0.00,0.00,0.00,1.00,0.00,0.00,11,79,1.00,0.00,0.09,0.04,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,210,151,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,8,89,1.00,0.00,0.12,0.04,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,212,786,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,8,99,1.00,0.00,0.12,0.05,0.00,0.00,0.00,0.00,normal.']

In [0]:
schema = spark.sparkContext.textFile("/FileStore/tables/schema-1.txt", 4)
schema = schema.map(lambda x: x.split(':')[0]).collect()

dataRDD = dataRDD.map(lambda x: x.split(","))

df = dataRDD.toDF(schema)

num_features = len(df.columns)
print("The total number of features present in the dataset: ", num_features)

The total number of features present in the dataset: 42

In [0]:
# Extract the columns
new_df = df.select(col("duration"), col("protocol_type"), col("service"),\
                   col("src_bytes"), col("dst_bytes"), col("flag"), col("label"))

#Build new RDD
new_rdd = df.rdd.map(list)

#print schema 
new_df.printSchema()

root
-- duration: string (nullable = true)
-- protocol_type: string (nullable = true)
-- service: string (nullable = true)
-- src_bytes: string (nullable = true)
-- dst_bytes: string (nullable = true)
-- flag: string (nullable = true)
-- label: string (nullable = true)

In [0]:
#display
new_df.show(10)

+--------+-------------+-------+---------+---------+----+-------+
duration|protocol_type|service|src_bytes|dst_bytes|flag| label|
+--------+-------------+-------+---------+---------+----+-------+
 0| tcp| http| 181| 5450| SF|normal.|
 0| tcp| http| 239| 486| SF|normal.|
 0| tcp| http| 235| 1337| SF|normal.|
 0| tcp| http| 219| 1337| SF|normal.|
 0| tcp| http| 217| 2032| SF|normal.|
 0| tcp| http| 217| 2032| SF|normal.|
 0| tcp| http| 212| 1940| SF|normal.|
 0| tcp| http| 159| 4087| SF|normal.|
 0| tcp| http| 210| 151| SF|normal.|
 0| tcp| http| 212| 786| SF|normal.|
+--------+-------------+-------+---------+---------+----+-------+
only showing top 10 rows

In [0]:
# Number of connections based on protocol type
new_df.groupBy('protocol_type').count().sort('count').show()

+-------------+------+
protocol_type| count|
+-------------+------+
 udp| 20354|
 tcp|190065|
 icmp|283602|
+-------------+------+

In [0]:
display(new_df.groupBy('protocol_type').count().sort('count'))

protocol_type,count
udp,20354
tcp,190065
icmp,283602


In [0]:
# Number of connections based on service
new_df.groupBy('service').count().sort('count').show()

+-----------+-----+
 service|count|
+-----------+-----+
 red_i| 1|
 pm_dump| 1|
 tftp_u| 1|
 tim_i| 7|
 X11| 11|
 urh_i| 14|
 IRC| 43|
 Z39_50| 92|
 netstat| 95|
 ctf| 97|
 kshell| 98|
 name| 98|
netbios_dgm| 99|
 http_443| 99|
 exec| 99|
 ldap| 101|
 pop_2| 101|
 link| 102|
 netbios_ns| 102|
 daytime| 103|
+-----------+-----+
only showing top 20 rows

In [0]:
display(new_df.groupBy('service').count().sort('count'))

service,count
tftp_u,1
red_i,1
pm_dump,1
tim_i,7
X11,11
urh_i,14
IRC,43
Z39_50,92
netstat,95
ctf,97


In [0]:
display(df.groupBy('flag').count().sort('count'))

flag,count
OTH,8
S3,10
RSTOS0,11
S2,24
S1,57
SH,107
RSTO,579
RSTR,903
REJ,26875
S0,87007


In [0]:
display(new_df.groupBy('label').count().sort('count'))

label,count
spy.,2
perl.,3
phf.,4
multihop.,7
ftp_write.,8
loadmodule.,9
rootkit.,10
imap.,12
warezmaster.,20
land.,21


In [0]:
display(df.groupBy('urgent').count().sort('count'))

urgent,count
3,1
2,1
1,2
0,494017


In [0]:
from pyspark.ml.feature import *
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [0]:
# Data Preprocessing
dataframe = new_df.withColumn('label', \
                              when(df["label"]== 'normal.', 'normal')\
                              .otherwise('attack'))

inputs = ["protocol_type", "service", "flag", "label"]
for column in inputs:
  indexer = StringIndexer(inputCol=column, outputCol=column+"_index") 
  dataframe = indexer.fit(dataframe).transform(dataframe)
  dataframe = dataframe.drop(column)
  
vecAssembler = VectorAssembler()\
                .setInputCols(["protocol_type_index", "service_index", "flag_index"])\
                .setOutputCol("features")
dataframe = vecAssembler.transform(dataframe)

In [0]:
# Logistic Regression
from pyspark.ml.classification import LogisticRegression

(trainset, testset) = dataframe.randomSplit([0.8, 0.2])

lr = LogisticRegression(maxIter=10, featuresCol='features', labelCol='label_index')

model = lr.fit(trainset)
predictions = model.transform(testset)

In [0]:
# Model Co-efficients
print("Coefficients: " + str(model.coefficients))
print("Intercept: " + str(model.intercept))

Coefficients: [6.366662417539792,0.013425997903220833,-3.5834759869567283]
Intercept: -4.7026527400321685

In [0]:
# Model Summary
trainingSummary = model.summary
accuracy = trainingSummary.accuracy
truePositiveRate = trainingSummary.weightedTruePositiveRate
fMeasure = trainingSummary.weightedFMeasure()
precision = trainingSummary.weightedPrecision
print("Accuracy: %s\nTPR: %s\nF-measure: %s\nPrecision: %s"
      % (accuracy, truePositiveRate, fMeasure, precision))

Accuracy: 0.9771388967582663
TPR: 0.9771388967582662
F-measure: 0.9770274338029279
Precision: 0.9769923480525117

In [0]:
# Evaluation on Test set
evaluator = MulticlassClassificationEvaluator(
    labelCol="label_index", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Accuracy = %g " % (accuracy))

Test Accuracy = 0.97734